# Given a latitude and longitude, provide the N Closest locaitons

Design a service that can be queried with a latitude and longitude, and return the N closest locations.

## Scope

### users

Lets assume this is for google maps, so 1 Billion+ Monthly active users

### use-cases

- what is a location?
(city, town, place of interest, ...), lets assume any of these.

- how do we define closeness?
in the simple case, we'll call it euclidian distance, but we could use something like manhattan distance/route-finding to determine distance as well.  Let's abstract the distance function for now, but treat it like euclidean distance.

- how do users use this
--given my position, give me the closest locations
--given a locations position, give me the closest locations
-given a random position, give me the closest locations

- additional functionality
We can discuss it later.


#### capacity
- back of envelope calculations

- reads/s
1 Billion active users / month
2.5 million sec/month
4000 reads / sec (avg) * (avg user reads / month)

- writes/s

writes are generated internally when new locations are added / locations are updated.  We can assume something like 10,000 new locations a month.

Additionally, we could allow for user based writes to locations, but this would require a security service, reviewers.




## high-level architecture

- client
- DNS/router
- load-balancer/router
- web-server (possibly shared)
- service endpoint

read-service & write service



- data layer (including caching)



everything down to the web-server should be handeled by maps.google.com, since it already exists, and we're just providing them a microservice.

We may already have a locations DB from maps, and we can create a truncated locations DB for our use.


## Service endpoint

We've defined our use-case as returning the N closest points, but there are similar use cases.  So we've defined a read only end-point with a single get command.  So we can define an endpoint that only responds to //gets

The 'get(latitude, longitude, N)' will take in two, optionally 3 values. N would represent the number of locations to return.

The response should be a ordered list of responses


## componant design

### OO design


#### objects

- locations

public:

getLocation(id), returns : location

candidate_locations_getter(pos), returns : [locations]

location_filter(pos, n, locations), returns: [locations]

- tiles

tile_getter_obj

getTile(pos), returns [tile]

getTilesWithNLocations(pos, N), returns : [tiles]

looks up ids with tileHashService

- tileHash

encode()
decode()

update()

- locationGetter

getNLocations(pos, n), returns [locations]

- locationWriter

writeLocation(location)

bulkWriteLocation([locations])

public:



location_writer(location)

bulk_location_writer([locations])

private:


- DB design

- precision

we need to decide some level of baseline precision for our request lat/long.  Our request should come in with the highest precision of GPS data available to it (clicking on a zoomed out map, not very precise, GPS locator on phone, very precise).  We should define our baseline precision.

The first step of a request will be to apply this truncation.

- Storage

Our storage strucutre will be highly coupled to our search strategy.  In 2D, we can tile our word along lat. and long.  The tiles will be stored with keys that are 2-D hashed 

- Tiles

(lat_block_idx, long_block_idx), neighbors, num_locations, [(location_ids, (lat., long))]

For each request, we should return 6 tiles, the hit-tile (where the requested position is) and the five sourounding tiles.

If the sum of their "num_locations" is greater than or equal to N, we collect all of the locations in the tiles and run through the locations in the 6 tiles, and collect the N closest with our distance function.  we can do this with a sort, or a priority queue and one pass.

- locations
Our locations table will be a NoSQL K-V type DB, allowing for complex schema, varying types

(location_id, meta_data)

## bottlenecks

Writes are somewhat trivial, so bottlenecks will be mostly be in read.  There will be periodicity in read volume across locations, and locations should have seperate caches.  Horizontal scaling, automatic scaling, and location specific caching should handle most of  this.  As far as prior knowledge of scaling requirements, we can get a lot of that info from historic maps.google traffic.


### maintaining hash-bin consistancy

hash-bins will only get smaller, and only when we do writes.  We should try to maintain our tile size so that they have a consistant number of locaitons within them.  If N is fixed, this may be a good number, but we should assume N could change.




## scaling